In [1]:
import pronouncing
import markovify
import random
import re
import os
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential, load_model, save_model

In [2]:
def create_network(depth):
    model = Sequential()
    model.add(LSTM(1, input_shape=(2, 2), return_sequences=True))
    for i in range(depth):	
        model.add(LSTM(2, return_sequences=True))
        model.add(Dropout(0.2))
    model.add(LSTM(1, return_sequences=True))
    model.summary()
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [3]:
#Markov Chain
def markov(input):
    ######
	read = open(input, "r", encoding='utf-8').read()
	text_model = markovify.NewlineText(read)
	return text_model

In [4]:
def syllables(line):
    count = 0
    vowels = 'aeiouy'
    words = line.split(' ')
    for word in words:
        word = word.lower().strip(".:;?!")        
        if len(word) > 0 and word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count += 1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le') and len(word) > 2 and word[-3] not in vowels:
            count += 1
    return count / maxsyllables

In [5]:
# Make index of words that rhyme with your word
def rhymeindex(lyrics):
	rhyme_master_list = []
	print ("Building list of rhymes:")
	for i in lyrics:
		word = re.sub(r"\W+", '', i.split(" ")[-1]).lower()
		rhymeslist = pronouncing.rhymes(word)
		rhymeslistends = []      
		for i in rhymeslist:
			rhymeslistends.append(i[-2:])
		try:
			rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
		except Exception:
			rhymescheme = word[-2:]
		rhyme_master_list.append(rhymescheme)
	rhyme_master_list = list(set(rhyme_master_list))
	reverselist = [x[::-1] for x in rhyme_master_list]
	reverselist = sorted(reverselist)
	rhymelist = [x[::-1] for x in reverselist]
	print("List of Sorted 2-Letter Rhyme Ends:")
	print(rhymelist)
	return rhymelist

In [6]:
# Make index of rhymes that you use
def rhyme(line, rhyme_list):
	word = re.sub(r"\W+", '', line.split(" ")[-1]).lower()
	rhymeslist = pronouncing.rhymes(word)
	rhymeslistends = []
	for i in rhymeslist:
		rhymeslistends.append(i[-2:])
	try:
			rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
	except Exception:
		rhymescheme = word[-2:]
	try:
		float_rhyme = rhyme_list.index(rhymescheme)
		float_rhyme = float_rhyme / float(len(rhyme_list))
		return float_rhyme
	except Exception:
		float_rhyme = None
		return float_rhyme

In [7]:
#Separate each line of the input txt
def split_lyrics_file(text_file):
	text = open(text_file, encoding='utf-8').read()
	text = text.split("\n")
	while "" in text:
		text.remove("")
	return text

In [8]:
#Generate lyrics
def generate_lyrics(text_model, text_file):
	bars = []
	last_words = []
	lyriclength = len(open(text_file,encoding='utf-8').read().split("\n"))
	count = 0
	markov_model = markov(text_file)
	while len(bars) < lyriclength / 9 and count < lyriclength * 2:
		bar = markov_model.make_sentence(max_overlap_ratio = .49, tries=100)
		if type(bar) != type(None) and syllables(bar) < 1:
			def get_last_word(bar):
				last_word = bar.split(" ")[-1]
				if last_word[-1] in "!.?,":
					last_word = last_word[:-1]
				return last_word
			last_word = get_last_word(bar)
			if bar not in bars and last_words.count(last_word) < 3:
				bars.append(bar)
				last_words.append(last_word)
				count += 1
	return bars

In [9]:
#Build dataset
def build_dataset(lines, rhyme_list):
	dataset = []
	line_list = []
	for line in lines:
		line_list = [line, syllables(line), rhyme(line, rhyme_list)]
		dataset.append(line_list)
	x_data = []
	y_data = []
	for i in range(len(dataset) - 3):
		line1 = dataset[i    ][1:]
		line2 = dataset[i + 1][1:]
		line3 = dataset[i + 2][1:]
		line4 = dataset[i + 3][1:]
		x = [line1[0], line1[1], line2[0], line2[1]]
		x = np.array(x)
		x = x.reshape(2,2)
		x_data.append(x)
		y = [line3[0], line3[1], line4[0], line4[1]]
		y = np.array(y)
		y = y.reshape(2,2)
		y_data.append(y)
	x_data = np.array(x_data)
	y_data = np.array(y_data)
	return x_data, y_data

In [10]:
#Compose verse
def compose_rap(lines, rhyme_list, lyrics_file, model):
	rap_vectors = []
	human_lyrics = split_lyrics_file(lyrics_file)
	initial_index = random.choice(range(len(human_lyrics) - 1))
	initial_lines = human_lyrics[initial_index:initial_index + 2]
	starting_input = []
	for line in initial_lines:
		starting_input.append([syllables(line), rhyme(line, rhyme_list)])
	starting_vectors = model.predict(np.array([starting_input]).flatten().reshape(1, 2, 2))
	rap_vectors.append(starting_vectors)
	for i in range(100):
		rap_vectors.append(model.predict(np.array([rap_vectors[-1]]).flatten().reshape(1, 2, 2)))
	return rap_vectors

In [11]:
#Compose verse (part 2)
def vectors_into_song(vectors, generated_lyrics, rhyme_list):
	print ("\n\n")
	print ("Writing verse:")
	print ("\n\n")
	def last_word_compare(rap, line2):
		penalty = 0
		for line1 in rap:
			word1 = line1.split(" ")[-1]
			word2 = line2.split(" ")[-1]
			while word1[-1] in "?!,. ":
				word1 = word1[:-1]
			while word2[-1] in "?!,. ":
				word2 = word2[:-1]
			if word1 == word2:
				penalty += 0.2
		return penalty
	def calculate_score(vector_half, syllables, rhyme, penalty):
		desired_syllables = vector_half[0]
		desired_rhyme = vector_half[1]
		desired_syllables = desired_syllables * maxsyllables
		desired_rhyme = desired_rhyme * len(rhyme_list)
		score = 1.0 - abs(float(desired_syllables) - float(syllables)) + abs(float(desired_rhyme) - float(rhyme)) - penalty
		return score
	dataset = []
	for line in generated_lyrics:
		line_list = [line, syllables(line), rhyme(line, rhyme_list)]
		dataset.append(line_list)
	rap = []
	vector_halves = []
	for vector in vectors:
		vector_halves.append(list(vector[0][0]))
		vector_halves.append(list(vector[0][1]))
	for vector in vector_halves:
		scorelist = []
		for item in dataset:
			line = item[0]
			if len(rap) != 0:
				penalty = last_word_compare(rap, line)
			else:
				penalty = 0
			total_score = calculate_score(vector, item[1], item[2], penalty)
			score_entry = [line, total_score]
			scorelist.append(score_entry)
		fixed_score_list = [0]
		for score in scorelist:
			fixed_score_list.append(float(score[1]))
		max_score = max(fixed_score_list)
		for item in scorelist:
			if item[1] == max_score:
				rap.append(item[0])
				print (str(item[0]))
				for i in dataset:
					if item[0] == i[0]:
						dataset.remove(i)
						break
				break
	return rap

In [12]:
def train(x_data, y_data, model):
	model.fit(np.array(x_data), np.array(y_data),
			  batch_size=2,
			  epochs=5,
			  verbose=1)

In [13]:
def train_model(train_mode,text_file,depth =4):
	model = create_network(depth)
	text_model = markov(text_file)
	if train_mode == True:
		bars = split_lyrics_file(text_file)
	if train_mode == False:
		bars = generate_lyrics(text_model, text_file)
	rhyme_list = rhymeindex(bars)
	if train_mode == True:
		x_data, y_data = build_dataset(bars, rhyme_list)
		train(x_data, y_data, model)
	return model

In [14]:
def main(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    for file_name in files:
        # Check if the item in the folder is a file
        if os.path.isfile(os.path.join(folder_path, file_name)):
            # Train the model for each file
            print(file_name)
            model = train_model(train_mode, os.path.join(folder_path, file_name))
    model.save('poem.h5')

In [15]:
# Set the desired values
depth = 4
maxsyllables = 16
train_mode = True
data = './data'

# Call the main function
main(data)

adele.txt
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 2, 1)              16        
                                                                 
 lstm_1 (LSTM)               (None, 2, 2)              32        
                                                                 
 dropout (Dropout)           (None, 2, 2)              0         
                                                                 
 lstm_2 (LSTM)               (None, 2, 2)              40        
                                                                 
 dropout_1 (Dropout)         (None, 2, 2)              0         
                                                                 
 lstm_3 (LSTM)               (None, 2, 2)              40        
                                                                 
 dropout_2 (Dropout)         (None, 2, 2)     


KeyboardInterrupt



In [ ]:
# Generate a poem from a given prompt
def generate_poem(prompt, model, text_model, rhyme_list, max_lines):
    # Generate initial lines using the Markov chain text model
    initial_lines = text_model.make_sentence().split()
    poem = initial_lines
    for _ in range(max_lines):
        # Prepare the input for the model
        x_data = []
        for i in range(len(poem) - 3):
            line1 = poem[i]
            line2 = poem[i + 1]
            x = [syllables(line1), rhyme(line1, rhyme_list),
                 syllables(line2), rhyme(line2, rhyme_list)]
            x_data.append(x)
        x_data = np.array(x_data)
        x_data = x_data.reshape(1, x_data.shape[0], x_data.shape[1])
        # Generate the next lines using the trained model
        predicted_lines = model.predict(x_data)
        # Select the line with the highest score
        selected_line = None
        highest_score = -1
        for line in predicted_lines[0]:
            score = line[0]
            if score > highest_score:
                selected_line = line[1]
                highest_score = score
        # Add the selected line to the poem
        poem.append(selected_line)
    # Return the generated poem as a string
    return ' '.join(poem)

In [ ]:
model = load_model('poem.h5')
prompt = "The sun sets low"
generated_poem = generate_poem(prompt, model, text_model, rhyme_list)
print(generated_poem)